# Showcase of fjsp-drl

This model requires python 3.6 and package versions in `requirements.txt`.
This model solves FJSP instances, which are a generalisation of JSSP.

In [1]:
import torch, json, PPO_model, copy, os, gym, time
import numpy as np
from env.load_data import nums_detec

# JSSP TO FJSP
First I need to convert JSSP to FJSP instance. Below is an example of JSSP instance in [Taillard's specification](http://jobshop.jjvh.nl/explanation.php)

```
6	6
1	3	6	7	3	6
8	5	10	10	10	4
5	4	8	9	1	7
5	5	5	3	8	9
9	3	5	4	3	1
3	3	9	10	4	1
3	1	2	4	6	5
2	3	5	6	1	4
3	4	6	1	2	5
2	1	3	4	5	6
3	2	5	6	1	4
2	4	6	1	5	3
```

Below is the same problem as [FJSP](https://people.idsia.ch/~monaldo/fjsp.html)

```
6   6   1   
6   1   3   1   1   1   3   1   2   6   1   4   7   1   6   3   1   5   6   
6   1   2   8   1   3   5   1   5   10  1   6   10  1   1   10  1   4   4   
6   1   3   5   1   4   4   1   6   8   1   1   9   1   2   1   1   5   7   
6   1   2   5   1   1   5   1   3   5   1   4   3   1   5   8   1   6   9   
6   1   3   9   1   2   3   1   5   5   1   6   4   1   1   3   1   4   1   
6   1   2   3   1   4   3   1   6   9   1   1   10  1   5   4   1   3   1   
```

Helper functions to convert JSSP to FJSP:

In [2]:
def parse_instance_taillard(filename):
    '''Parses instance written in Taillard specification: http://jobshop.jjvh.nl/explanation.php
    
      Args:
        filename - file containing the instance in Taillard specification

      Returns:
        number of jobs,
        number of machines,
        the processor times for each operation,
        the order for visiting the machines
    '''

    with open(filename, 'r') as f:
        # parse number of jobs J and machines M
        J, M = map(int, f.readline().split())

        # Initialize two empty numpy arrays with dimensions J x M
        processor_times = np.empty((J, M), dtype=int)
        orders_of_machines = np.empty((J, M), dtype=int)
    
        # Read the next J lines containing processor times
        for i in range(J):
            processor_times[i] = list(map(int, f.readline().split()))
    
        # Read the next J lines containing orders of machines
        for i in range(J):
            orders_of_machines[i] = list(map(int, f.readline().split()))

        return J, M, processor_times, orders_of_machines

def jssp_taillard_to_fjsp(filename: str) -> str:
    '''Transforms JSSP instance in Taillard's specification to FJSP instance
       and stores it in a temporary file
    
      Args:
        filename - name of the file with JSSP instance in Taillard's specification
        
      Returns:
        string - filename of the equivalent FJSP instance 
    '''
    # parse JSSP Taillard instance
    J, M, processor_times, orders_of_machines = parse_instance_taillard(filename)
    
    # convert JSSP to FJSP
    with open("/tmp/fjsp_" + filename.split("/")[-1], 'w') as f:
        # write number of jobs, number of machines, and jobs/machines (which is always 1 for JSSP)
        f.write(str(J) + "   " + str(M) + "   1\n")
        
        # each line is a job
        for i in range(J):
            # each line starts with the number of operations in a job
            number_of_operations = len(processor_times[i])
            f.write(str(number_of_operations) + "  ")
            
            # print the operation as a tuple (number of available machines, current machine, processing time)
            for j in range(number_of_operations):
                f.write(" 1   " + str(orders_of_machines[i][j]) + "   " + str(processor_times[i][j]) + "  ")
                
            f.write('\n')
            
    return "/tmp/fjsp_" + filename.split("/")[-1]
    

# Model

## Model setup

Setup torch, load the configuration of the experiment prepare the model and load its parameters from the checkpoint

In [3]:
# setup torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device.type=='cuda':
    torch.cuda.set_device(device)
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')
    
# load configuration of the experiment
with open("./config.json", 'r') as load_f:
    load_dict = json.load(load_f)
    
model_paras = load_dict['model_paras']
model_paras["actor_in_dim"] = model_paras["out_size_ma"] * 2 + model_paras["out_size_ope"] * 2
model_paras["critic_in_dim"] = model_paras["out_size_ma"] + model_paras["out_size_ope"]
model_paras['device'] = device
env_paras = load_dict['env_paras']
env_paras['device'] = device
env_paras['batch_size'] = 1

# create model and its parameters from the checkpoint
memories = PPO_model.Memory()
model_CKPT = torch.load('./model/save_10_5.pt')
model = PPO_model.PPO(model_paras, load_dict['train_paras'])
model.policy.load_state_dict(model_CKPT)
model.policy_old.load_state_dict(model_CKPT)

<All keys matched successfully>

In [4]:
def solve_instance(instance_file, model, memories, flag_sample=False):
    '''Solves FJSP instance using given model and memories
    
      Args:
          instance_file - name of the file with the FJSP instance
          model - model used to solve the instance
          memories - model's memories
    
      Returns:
          makespan and time it took to solve the instance
    '''
    with open(instance_file, 'r') as file_object:
        # load the parameters of the instance
        line = file_object.readlines()
        ins_num_jobs, ins_num_mas, _ = nums_detec(line)
        env_paras["num_jobs"] = ins_num_jobs
        env_paras["num_mas"] = ins_num_mas
        
        # create env and get states and completion signal
        env = gym.make('fjsp-v0', case=[instance_file], env_paras=env_paras, data_source='file')
        state = env.state
        dones = env.done_batch
        done = False  # Unfinished at the beginning
        
        # perform scheduling
        start = time.time()
        while not done:
            with torch.no_grad():
                actions = model.policy_old.act(state, memories, dones, flag_sample=flag_sample, flag_train=False)
            state, rewards, dones = env.step(actions)  # environment transit
            done = dones.all()
        run_time = time.time() - start  # The time taken to solve this environment (instance)

        gantt_result = env.validate_gantt()[0]
        if not gantt_result:
            raise Exception("Scheduling error")

        return copy.deepcopy(env.makespan_batch), run_time

## FJSP instances from the paper

Run the experiment on some instances from this paper.

In [5]:
TEST_FILES_PATH = "data_test/Public/"
test_files = sorted(os.listdir(TEST_FILES_PATH))
for file in test_files[:10]:
    makespan, _ = solve_instance(TEST_FILES_PATH + file, model, memories)
    print("Makespan of", file, ":", makespan.item())

Makespan of Mk01.fjs : 43.0
Makespan of Mk02.fjs : 38.0
Makespan of Mk03.fjs : 204.0
Makespan of Mk04.fjs : 81.0
Makespan of Mk05.fjs : 192.0
Makespan of Mk06.fjs : 99.0
Makespan of Mk07.fjs : 221.0
Makespan of Mk08.fjs : 529.0
Makespan of Mk09.fjs : 339.0
Makespan of Mk10.fjs : 264.0


## JSSP instances from our benchmarks

Run the experiment on our benchmarks

In [6]:
def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../benchmarks/"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

In [7]:
instances = get_all_instances_in_taillard_specification()
for instance in instances:
    fjsp_instance = jssp_taillard_to_fjsp(instance)
    makespan, _ = solve_instance(fjsp_instance, model, memories)
    print("Makespan of", instance.split("/")[-1], ":", makespan.item())

Makespan of yn03.txt : 1073.0
Makespan of yn02.txt : 1043.0
Makespan of yn01.txt : 1032.0
Makespan of yn04.txt : 1247.0
Makespan of swv14.txt : 3955.0
Makespan of swv18.txt : 2852.0
Makespan of swv20.txt : 2823.0
Makespan of swv17.txt : 2794.0
Makespan of swv19.txt : 3009.0
Makespan of swv08.txt : 2401.0
Makespan of swv02.txt : 2098.0
Makespan of swv05.txt : 1842.0
Makespan of swv12.txt : 4036.0
Makespan of swv01.txt : 1971.0
Makespan of swv06.txt : 2100.0
Makespan of swv11.txt : 3961.0
Makespan of swv13.txt : 4316.0
Makespan of swv03.txt : 1919.0
Makespan of swv07.txt : 2070.0
Makespan of swv09.txt : 2216.0
Makespan of swv04.txt : 1905.0
Makespan of swv15.txt : 3903.0
Makespan of swv10.txt : 2297.0
Makespan of swv16.txt : 2924.0
Makespan of ta32.txt : 2160.0
Makespan of ta11.txt : 1629.0
Makespan of ta79.txt : 5539.0
Makespan of ta60.txt : 3014.0
Makespan of ta28.txt : 1886.0
Makespan of ta56.txt : 3151.0
Makespan of ta65.txt : 3218.0
Makespan of ta43.txt : 2341.0
Makespan of ta40.txt

KeyboardInterrupt: 